In [1]:
'''
title           : blockchain_client.py
description     : A blockchain client implemenation, with the following features
                  - Wallets generation using Public/Private key encryption (based on RSA algorithm)
                  - Generation of transactions with RSA encryption      
author          : Ajay Malhotra
date_created    : 20190423
date_modified   : 20190423
version         : 0.3
usage           : python blockchain_client.py
                  python blockchain_client.py -p 8080
                  python blockchain_client.py --port 8080
python_version  : 3.7.2
Comments        : Wallet generation and transaction signature is based on [1]
References      : [1] https://github.com/julienr/ipynb_playground/blob/master/bitcoin/dumbcoin/dumbcoin.ipynb
                  [2] https://github.com/adilmoujahid/blockchain-python-tutorial/blob/master/blockchain_client/blockchain_client.py
'''
from collections import OrderedDict

import binascii

import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5

import requests
from flask import Flask, jsonify, request, render_template

In [2]:
class Transaction:

    def __init__(self, sender_address, sender_private_key, recipient_address, value):
        self.sender_address = sender_address
        self.sender_private_key = sender_private_key
        self.recipient_address = recipient_address
        self.value = value

    def __getattr__(self, attr):
        return self.data[attr]

    def to_dict(self):
        return OrderedDict({'sender_address': self.sender_address,
                            'recipient_address': self.recipient_address,
                            'value': self.value})

    def sign_transaction(self):
        """
        Sign transaction with private key
        """
        private_key = RSA.importKey(binascii.unhexlify(self.sender_private_key))
        signer = PKCS1_v1_5.new(private_key)
        h = SHA.new(str(self.to_dict()).encode('utf8'))
        return binascii.hexlify(signer.sign(h)).decode('ascii')

In [3]:
app = Flask(__name__)

@app.route('/')
def index():
    return render_template('./index.html')

@app.route('/make/transaction')
def make_transaction():
    return render_template('./make_transaction.html')

@app.route('/view/transactions')
def view_transaction():
    return render_template('./view_transactions.html')

@app.route('/wallet/new', methods=['GET'])
def new_wallet():
    random_gen = Crypto.Random.new().read
    private_key = RSA.generate(1024, random_gen)
    public_key = private_key.publickey()
    response = {
        'private_key': binascii.hexlify(private_key.exportKey(format='DER')).decode('ascii'),
        'public_key': binascii.hexlify(public_key.exportKey(format='DER')).decode('ascii')
    }
    return jsonify(response), 200


In [4]:
@app.route('/generate/transaction', methods=['POST'])
def generate_transaction():

    sender_address = request.form['sender_address']
    sender_private_key = request.form['sender_private_key']
    recipient_address = request.form['recipient_address']
    value = request.form['amount']

    transaction = Transaction(sender_address, sender_private_key, recipient_address, value)

    response = {'transaction': transaction.to_dict(), 'signature': transaction.sign_transaction()}

    return jsonify(response), 200


In [ ]:
if __name__ == '__main__':
    '''from argparse import ArgumentParser

    parser = ArgumentParser()
    parser.add_argument('-p', '--port', default=8080, type=int, help='port to listen on')
    args = parser.parse_args()
    port = args.port
    '''
    port = 8080
    app.run(host='127.0.0.1', port=port)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Apr/2019 19:29:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2019 19:29:37] "GET /static/css/custom.css HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2019 19:29:37] "GET /static/vendor/bootstrap/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2019 19:29:37] "GET /static/vendor/jquery/jquery.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2019 19:29:37] "GET /static/vendor/bootstrap/js/bootstrap.bundle.min.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2019 19:29:37] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Apr/2019 19:29:40] "GET /wallet/new HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2019 19:31:02] "GET /make/transaction HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2019 19:31:02] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Apr/2019 19:31:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2019 19:31:34] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2019 19:31:34] "GET /static/vendor/jquery/jquery.min.js HTTP